In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import argparse
import locale
import glob
import cv2
import os
from sklearn.metrics import r2_score

In [ ]:
import tensorflow as tf
from tensorflow import keras
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
from sklearn.preprocessing import StandardScaler
import tensorflow_datasets as tfds

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
c = np.load('path/image_data_211106.npy')

In [ ]:
c = c/255.0

In [ ]:
c = c.astype(np.float32)
c.dtype

dtype('float32')

In [ ]:
batch_size, height, width, channels = c.shape

In [ ]:
result_df = pd.read_excel('path/211108_Log_score.xlsx')

In [ ]:
#result_df

In [ ]:
d = result_df['Log_score'].to_numpy()

In [ ]:
# 축 바꿈
d = d.reshape(-1, 1)

In [ ]:
# y 값 스케일
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
y = scaler.fit_transform(d)

In [ ]:
# #mms
# from sklearn.preprocessing import MinMaxScaler

# scaler=MinMaxScaler()
# y = scaler.fit_transform(d)

In [ ]:
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()
# y = scaler.fit_transform(d)

In [ ]:
import random

minb = []

for i in range(1100):
  i = random.randint(0,7502)
  while i in minb:
    i = random.randint(0,7502)
  minb.append(i)

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(minb)):
  k = minb[i]
  if i<1001:
    xt = c[k]
    yt = y[k]
    X_train.append(xt)
    y_train.append(yt)
  elif i>=1001:
    xtt = c[k]
    ytt = y[k]
    X_test.append(xtt)
    y_test.append(ytt)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
from collections import Counter
Counter(minb)

In [ ]:
X_train = c[:2000]
X_test = c[2000:]

In [ ]:
y_train = y[:2000]
y_test = y[2000:]

In [ ]:
model_cnn = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="ELU", padding="same", input_shape=[202, 360, 3]),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="ELU", padding="same"),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="ELU", padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation="ELU", padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation="ELU", padding="same"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="ELU"),
    keras.layers.Dense(64, activation="ELU"),
    keras.layers.Dense(10, activation="ELU"),
    keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 202, 360, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 101, 180, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 101, 180, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 101, 180, 128)     147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 101, 180, 256)     295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 101, 180, 256)     590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 90, 256)       0

In [ ]:
my_adam = keras.optimizers.Adam(learning_rate=0.01)
model_cnn.compile(loss="mse",
                 optimizer ='sgd',
                 metrics=['accuracy'])

history = model_cnn.fit(X_train, y_train, epochs=10)

In [ ]:
k = model_cnn.predict(X_test)
print(k[:10],'mmm', y_test[:10])

In [ ]:
results = model_cnn.evaluate(X_train, y_train)
print(results)

63/63 [==============================] - 33s 522ms/step - loss: 0.0110 - accuracy: 5.0000e-04
[0.010978487320244312, 0.0005000000237487257]


In [ ]:
#rmse 는 평균 제곱근 오차로 추정값 또는 모델이 예측한 값과 실제 환경에서 관촬된 값의 실제 차이를 나타낼 때 사용되는 측도
pred = np.squeeze(model_cnn.predict(X_test))
answ = y_test

rmse = np.sqrt(model_cnn.evaluate(X_test,verbose=0))
r2 = r2_score(pred, answ)
#print("     Test RMSE: {:.5f}".format(rmse))
print("Test R^2 Score: {:.5f}".format(r2))

Test R^2 Score: 0.14055


In [ ]:
#모델이 데이터에 얼마나 적합한지 알기 위해 r2_score 가 1에 가까울수록 좋다고 평가하면됨
predicted_ages = np.squeeze(model.predict(X_test))
true_ages = y_test

rmse = np.sqrt(model.evaluate(X_test, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

     Test RMSE: 0.00000
Test R^2 Score: 0.08119
